In [ ]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [ ]:
# Loading modules that we need
import unittest
from pyspark.sql.dataframe import DataFrame
from typing import Any

In [ ]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: str) -> DataFrame:
    """
    Arguments:
        table_name: name of the table to load
    """
    return spark.read.parquet(table_name)

users_df = load_df("/user/hive/warehouse/users")
comments_df = load_df("/user/hive/warehouse/comments")
posts_df = load_df("/user/hive/warehouse/posts")

#### Subtask 1: implementing two helper functions
Impelment these two functions:
1. 'run_query' that gets a Spark SQL query and run it on df which is a Spark DataFrame; it returns the content of the first column of the first row of the DataFrame that is the output of the query;
2. 'run_query2' that is similar to 'run_query' but instead of one DataFrame gets two; it returns the content of the first column of the first row of the DataFrame that is the output of the query.

Note that the result of a Spark SQL query is itself a Spark DataFrame.

In [ ]:
def run_query(query: str, df: DataFrame) -> Any:
    """
    Arguments:
        query: a SQL query string
        df: the DataFrame that the query will be executed on
    """
    df.createOrReplaceTempView("df")
    query_result = spark.sql(query)
    return query_result.collect()[0][0]

def run_query2(query: str, df1: DataFrame, df2: DataFrame) -> Any:
    """
    Arguments:
        query: a SQL query string
        df1: DataFrame A
        df2: DataFrame B
    """
    df1.createOrReplaceTempView("df1")
    df2.createOrReplaceTempView("df2")
    query_result = spark.sql(query)
    return query_result.collect()[0][0]

In [ ]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


#### Subtask 2: writing a few queries
Write the following queries in SQL to be executed by Spark in the next cell.

1. 'q1': find the 'Id' of the most recently created post ('df' is 'posts_df') 
2. 'q2': find the number users
3. 'q3': find the 'Id' of the user who posted most number of answers
4. 'q4': find the number of questions
5. 'q5': find the display name of the user who posted most number of comments

Note that 'q1' is already available below as an example. Moreover, remmebr that Spark supports ANSI SQL 2003 so your queries have to comply with that standard.

In [ ]:
# Find the 'Id' of the most recently created post
# 'df' is 'posts_df'
q1 = "SELECT * FROM df ORDER BY CreationDate DESC limit 1"

# Find the number of users
# 'df' is 'users_df'
q2 = "SELECT COUNT(*) FROM df"

# Find the 'Id' of the user who posted most number of answers
# 'df' is 'posts_df'
# 'PostTypeId = 2' is an answer post
q3 = """
    SELECT OwnerUserId, COUNT(Id)
    FROM df
    WHERE PostTypeId = 2
    GROUP BY OwnerUserId
    ORDER BY COUNT(Id) DESC
    LIMIT 1
"""

# Find the number of questions
# 'df' is 'posts_df'
q4 = """
    SELECT COUNT(Id)
    FROM df
    WHERE PostTypeId = 1
"""

# Find the display name of the user who posted most number of comments
# 'df1' is 'users_df'
# 'df2' is 'comments_df'
q5 = """
    SELECT DisplayName
    FROM df1
    WHERE Id = (
      SELECT UserId
      FROM df2
      GROUP BY df2.UserId
      ORDER BY COUNT(*) DESC
      LIMIT 1
    );
"""

#### Subtask 3: validating the implementations by running the tests

Run the cell below and make sure that all the tests run successfully.

In [ ]:
%%unittest_main
class TestTask2(unittest.TestCase):
    def test_q1(self):
        # find the id of the most recent post
        r = run_query(q1, posts_df)
        self.assertEqual(r, 95045)

    def test_q2(self):
        # find the number of the users
        r = run_query(q2, users_df)
        self.assertEqual(r, 91616)

    def test_q3(self):
        # find the user id of the user who posted most number of answers
        r = run_query(q3, posts_df)
        self.assertEqual(r, 64377)

    def test_q4(self):
        # find the number of questions
        r = run_query(q4, posts_df)
        self.assertEqual(r, 28950)

    def test_q5(self):
        # find the display name of the user who posted most number of comments
        r = run_query2(q5, users_df, comments_df)
        self.assertEqual(r, "Neil Slater")

Success

.....
----------------------------------------------------------------------
Ran 5 tests in 7.437s

OK
Out[26]: <unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please answer the following questions. Write your answer in one to two short paragraphs. Don't copy-paste; instead, write your own understanding.

1. What is the difference between 'DataFrame', 'Dataset', and 'Resilient Distributed Datasets (RDD)'? 
2. When do you suggest using RDDs instead of using DataFrames?
3. What is the main benefit of using DataSets instead of DataFrames?

Write your answers in the next cell.

#### 1. DataFrames, Datasets and Resilient Distributed Datasets (RDDs)

The key difference between RDDs on one hand, and DataFrames/Datasets on the other hand, is that an RDD treats its data as **unstructured**, whereas a DataFrame/Dataset has a **schema** that defines **named columns** for its data. Datasets are essentially a more **type-safe** version of DataFrames. A `Dataset<T>` is a dataset with elements of type `T`, and in Spark 2.0, a DataFrame is an alias for `Dataset<Row>`, where `Row` is an untyped object (though still with a defined schema at runtime).

The defined schema of DataFrames and Datasets allow the use of **Spark SQL**, so users may write SQL queries instead of using Spark methods. The Spark SQL engine also includes an **optimizer**, which can drastically improve performance of operations on these DataFrames/Datasets.

#### 2. When to use RDDs instead of DataFrames

RDDs are preferable to DataFrames when the **schema of the data is unknown**. In this sense, RDDs may be more flexible, as they do not require the user to define the schema of the data. RDDs should likely not be used instead of DataFrames if the data _has_ a known schema and the system can benefit from the improved performance of DataFrames' optimizer.

#### 3. Benefits of using Datasets instead of DataFrames

Datasets provide better **compile-time type safety** than DataFrames. The generic type `T` of `Dataset<T>` (as described in answer 1) allows the compiler to check that the Dataset API is used correctly according to the data type. This obviously only works for statically typed compiled languages – in Python, DataFrames are used where Scala/Java could use Datasets, since Python does not have built-in compile-time type checking.